### 저장된 모델 로드 & 활용
- 모델 파일 종류	
	* 가중치 및 절편 저장 파일	=> 동일한 구조의 모델 인스턴스 생성 필요
	* 모델 전체 저장 파일		=> 바로 로딩 후 사용 가능

[1] 모듈로딩 <hr>

In [1]:
# 모듈 로딩
# - Model 실행 관련
import torch              
import torch.nn as nn              
import torch.nn.functional as F 
from torchinfo import summary 

# - 그 외
from RNN_utils import *
import os
import pickle

In [2]:
## 모델 파일 관련
# 모델 파일 저장
import os

# 저장경로
SAVE_PATH= r'C:\Users\KDP-43\Desktop\NLTK_project\models'

# 저장 파일명
# SAVE_MODEL = SAVE_PATH+'\model_num_loss(1.1301)_score(0.6264).pth'
SAVE_MODEL = SAVE_PATH+'\model_num_loss(0.7953)_score(0.6651).pth'


[2] 모델 로딩 - 모델 전체 파일 사용

In [3]:
if os.path.exists(SAVE_MODEL):
	Model = torch.load(SAVE_MODEL, weights_only=False)
else:
	print(f'{SAVE_MODEL} 파일이 존재하지 않습니다. 다시 확인하세요.')

In [4]:
summary(Model)

Layer (type:depth-idx)                   Param #
SentenceClassifier                       --
├─Embedding: 1-1                         640,256
├─LSTM: 1-2                              8,832
├─Linear: 1-3                            17
├─Dropout: 1-4                           --
Total params: 649,105
Trainable params: 649,105
Non-trainable params: 0

[3] 예측 <hr>

In [5]:
# data = [ float(x) for x in input("SL, SW, PL, PW: ").split(",") ]
# data = "이쯤되면 한국영화 장르에 '신파'를 추가해야할 것 같다."
# data = "범죄 영화에 ‘유튜브에 미친 사람’ 좀 그만 넣었으면…"
# data = "묘하게 누아르 감성이 있어서 신기하네. 1이랑 비교도 할 수 없이 좋다. 좀 과장하면 흔히들 범죄도시류 형사물에 머물던 장르에서 새 계보를 열어낸 것만 같다"
# data = "취향저격 당했다 눈물 펑펑 중반부에 10분 정도 단축시키는 빠른 전개를 보여줬다면 5점을 줬을텐데 ㅠㅠ"


# 논외 data
# data = "열등의식과 무력감이 빚어낸 기묘한 리듬감.  봉준호 감독 스타일의 정점!"
data = "야만과 무능의 그 겨울밤에 대한 분노가 시종 팽팽한 긴장감 속에서 펄펄 끓는다."
cleaned = cleantext(data)
print(cleaned)

야만과 무능의 그 겨울밤에 대한 분노가 시종 팽팽한 긴장감 속에서 펄펄 끓는다


In [6]:
# 불용어 처리 데이터 로드
STOP_FILE = './kor_stopwordsVer2.txt'

kor_stopwords = clean_korStopwords(STOP_FILE)

kor_stopwords.append('.')
kor_stopwords.append('\n')
kor_stopwords.append('\n ')
kor_stopwords.append('영화')
kor_stopwords.append('은')
kor_stopwords.append('는')
kor_stopwords.append('유아인')
kor_stopwords.append('류승완')
kor_stopwords.append('황정민')
kor_stopwords.append('이순신')


In [7]:
# 단어사전 로드
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

print("단어 사전이 불러와졌습니다.")
print(vocab)

단어 사전이 불러와졌습니다.
{'<pad>': 0, '<unk>': 1, '한': 2, '도': 3, '적': 4, '연기': 5, '잘': 6, '다': 7, '수': 8, '하는': 9, '만': 10, '생각': 11, '너무': 12, '인': 13, '배우': 14, '감독': 15, '더': 16, '이다': 17, '액션': 18, '한국': 19, '하고': 20, '할': 21, '사람': 22, '코미디': 23, '고': 24, '못': 25, '게': 26, '안': 27, '내': 28, '신파': 29, '캐릭터': 30, '있는': 31, '말': 32, '원작': 33, '역사': 34, '보다': 35, '한다': 36, '없는': 37, '진짜': 38, '없이': 39, '점': 40, '스토리': 41, '정말': 42, '정도': 43, '같은': 44, '두': 45, '장면': 46, '눈물': 47, '연출': 48, '봄': 49, '보는': 50, '느낌': 51, '지': 52, '전': 53, '했다': 54, '현실': 55, '랑': 56, '재미': 57, '된': 58, '보고': 59, '볼': 60, '그냥': 61, '관객': 62, '본': 63, '없다': 64, '이야기': 65, '님': 66, '마지막': 67, '끝': 68, '이렇게': 69, '속': 70, '감동': 71, '대사': 72, '좋은': 73, '작품': 74, '듯': 75, '오락': 76, '중': 77, '때문': 78, '화': 79, '서': 80, '거': 81, '웃음': 82, '많이': 83, '해': 84, '신': 85, '되는': 86, '인물': 87, '알': 88, '지옥': 89, '부분': 90, '베테': 91, '인가': 92, '이라는': 93, '라는': 94, '결말': 95, '모든': 96, '역시': 97, '최고': 98, '건': 99, '한국영': 100, '가장':

In [8]:
# 토크나이저 인스턴스 생성
tokenizer = Okt()

tokenList = [ token for token in tokenizer.morphs(cleaned, norm=False, stem=False) if token not in kor_stopwords]
print(tokenList)

['야만', '무능', '겨울', '밤', '대한', '분노', '시종', '팽팽', '한', '긴장감', '속', '펄펄', '끓는다']


In [9]:
for idx, token in enumerate(tokenList):
    if token not in vocab.keys():
        tokenList[idx] = '<unk>'
        
print(tokenList)

['야만', '무능', '겨울', '밤', '대한', '분노', '<unk>', '팽팽', '한', '긴장감', '속', '<unk>', '<unk>']


In [10]:
# 사전을 사용해 토큰을 정수 인덱스로 변환
max_length = 35

indices = [vocab[token] for token in tokenList]
# 최대 길이를 맞추기 위해 패딩 처리
if len(indices) < max_length:
	indices += [vocab['<pad>']] * (max_length - len(indices))
else:
	indices = indices[:max_length]
# 파이토치 텐서로 변환
input_tensor = torch.tensor(indices).unsqueeze(0)  # (1, max_length)

In [11]:
# 예측
Model.eval()

with torch.no_grad():
    
	# 추론/평가
    pre_y = Model(input_tensor)
    print(pre_y)
    
	# 예측 결과 처리 (logits를 확률로 변환)
    probability = torch.sigmoid(pre_y).item()
    
    # 결과 출력 (임계값을 0.5로 설정해 긍정/부정 분류)
    if probability > 0.5:
        result =  "긍정", probability
    else:
        result =  "부정", probability
        
print(result)

tensor([[1.6860]])
('긍정', 0.8437026143074036)
